In [1]:
#import cac thu vieb can thiet
from tensorflow import keras
from tensorflow.keras import layers
import os
import numpy as np
import cv2
from glob import glob
from sklearn import model_selection
import tensorflow as tf
from tensorflow.keras.metrics import Precision, Recall, MeanIoU
import matplotlib.pyplot as plt
from tensorflow.keras.utils import CustomObjectScope
from tqdm import tqdm
from tensorflow.keras.optimizers import Adam, Nadam, SGD
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from matplotlib import pyplot as plt

In [2]:
size_unet=256

In [3]:
def conv_block(x, num_filters):
    x = Conv2D(num_filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def build_model():
    size = 256
    num_filters = [64, 128, 256, 512, 1024]
    inputs = keras.Input((size_unet, size_unet, 3))

    skip_x = []
    x = inputs
    ## Encoder
    for f in num_filters:
        x = conv_block(x, f)
        skip_x.append(x)
        x = MaxPool2D((2, 2))(x)

    ## Bridge
    x = conv_block(x, num_filters[-1])

    num_filters.reverse()
    skip_x.reverse()
    ## Decoder
    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2))(x)
        xs = skip_x[i]
        x = Concatenate()([x, xs])
        x = conv_block(x, f)

    ## Output
    x = Conv2D(1, (1, 1), padding="same")(x)
    x = Activation("sigmoid")(x)

    return Model(inputs, x)

    model = build_model()
    model.summary()

In [4]:
model = build_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [5]:
#Ham danh gia dice coef
smooth = 1.
def dice_coef(y_true, y_pred):
    y_true_f = tf.keras.layers.Flatten()(y_true)
    y_pred_f = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)


def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

with CustomObjectScope({'dice_loss': dice_loss, 'dice_coef': dice_coef}):
  model = tf.keras.models.load_model("model.h5")

In [ ]:
def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (256, 256))
    x = x/255.0
    return x
def mask_parse(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask
def predict_mask(path):
    x=read_image(path)
    y_pred = model.predict(np.expand_dims(x, axis=0))[0] >0.5
    y_pred = mask_parse(y_pred) *255
    return (y_pred[:,:,0]).astype(np.uint8)
def predict_final(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    w=img.shape[0]
    h=img.shape[1]
    img = cv2.resize(img, (256, 256))
    mask=(predict_mask(path))
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours (img, contours, -1, (255,0,0), 2) 
    return img


In [ ]:
path="polyp_test_mang.jpg"
img_p=predict_final(path)
cv2.imshow("Anhdudoan",img_p)
cv2.waitKey(0) 
cv2.destroyAllWindows()
